## Helmholtz Boundary data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv

# Define the parameters of the problem
k = 20  # wavenumber
xi = 1  # xi

# Define the Lipschitz domain
x = np.linspace(0, 1, 501)
y = np.linspace(-0.5, 0.5, 501)
X, Y = np.meshgrid(x, y)

def cart2pol(x, y):
    R = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(R, phi)

def analytical_solution(x, y, k, order=1):
  (r,theta) = cart2pol(x,y)
  u_star = jv(order, k * r) * np.exp(1j * order * theta)
  return u_star

R,THETA = cart2pol(X,Y)

# Compute the solution
U = jv(xi, k * R) * np.exp(1j * xi * THETA)
# U = analytical_solution(X,Y,k)
# Plot the solution
fig, ax = plt.subplots(figsize=(8, 4))
im = ax.imshow(np.real(U), extent=[0, 1, -0.5, 0.5], cmap='viridis')
fig.colorbar(im)
ax.set_title('Analytical Solution of Circular Waves')
plt.show()

In [ ]:
def create_training_data(Nf, Ng, x_min, x_max, y_min, y_max):
    # Create random training points in the domain
    Xf = torch.rand(Nf, 2)  # Randomly sample Nf points
    Xf[:, 0] = Xf[:, 0] * (x_max - x_min) + x_min
    Xf[:, 1] = Xf[:, 1] * (y_max - y_min) + y_min
    Xf.requires_grad = True

    # Create uniform boundary points
    Xg = []
    x_boundary = np.linspace(x_min, x_max, Ng)
    y_boundary = np.linspace(y_min, y_max, Ng)

    for x in x_boundary:
        Xg.append([x, y_min])
        Xg.append([x, y_max])

    for y in y_boundary[1:-1]:
        Xg.append([x_min, y])
        Xg.append([x_max, y])

    Xg = torch.tensor(Xg, dtype=torch.float32)
    Xg.requires_grad = True

    return Xf, Xg


In [ ]:
def loss_function(tann, Xf, Xg, k, analytical_solution):
    # Evaluate the model on the training and boundary points
    u_f = tann(Xf)
    u_g = tann(Xg)

    # Calculate the Helmholtz equation residuals
    u_f_x = torch.autograd.grad(u_f, Xf, torch.ones_like(u_f), create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_f_xx = torch.autograd.grad(u_f_x[:, 0], Xf, torch.ones_like(u_f_x[:, 0]), create_graph=True, retain_graph=True, only_inputs=True)[0][:, 0]
    u_f_yy = torch.autograd.grad(u_f_x[:, 1], Xf, torch.ones_like(u_f_x[:, 1]), create_graph=True, retain_graph=True, only_inputs=True)[0][:, 1]

    laplacian_u = u_f_xx + u_f_yy
    equation_residuals = laplacian_u + k**2 * u_f.view(-1)

    # Calculate boundary condition residuals
    Xg_np = Xg.detach().numpy()
    g = analytical_solution(Xg_np[:, 0], Xg_np[:, 1], k)
    g = torch.tensor(np.real(g), dtype=torch.float32).view(-1, 1)

    boundary_residuals = u_g - g

    # Combine residuals and return the total loss
    loss = torch.mean(equation_residuals**2) + torch.mean(boundary_residuals**2)
    return loss

## TANN Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class TANN(nn.Module):
    def __init__(self, input_size, hidden_units, output_size, hidden_layers):
        super(TANN, self).__init__()
        self.input_size = input_size
        self.hidden_units = hidden_units
        self.output_size = output_size
        self.hidden_layers = hidden_layers

        layers = []
        layers.append(nn.Linear(self.input_size, self.hidden_units))
        layers.append(nn.Tanh())

        for _ in range(self.hidden_layers - 1):
            layers.append(nn.Linear(self.hidden_units, self.hidden_units))
            layers.append(nn.Tanh())

        layers.append(nn.Linear(self.hidden_units, self.output_size))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
def train_tann(tann, Xf, Xg, k, analytical_solution, epochs=50000, lr=1e-3, stop_criterion=2e-16):
    optimizer = optim.LBFGS(tann.parameters(), lr=lr, max_iter=50)

    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = loss_function(tann, Xf, Xg, k, analytical_solution)
            loss.backward()
            return loss

        optimizer.step(closure)

        # Check stopping criterion
        gradients = torch.cat([param.grad.view(-1) for param in tann.parameters()])
        max_gradient = torch.max(torch.abs(gradients)).item()
        if max_gradient < stop_criterion:
            break
        
#         if epoch % 1000 == 0:
#             loss = loss_function(tann, Xf, Xg, k, analytical_solution)
#             print("Epoch:", epoch, "Loss:", loss.item())

    return tann


## SIREN Model

In [ ]:
class SIREN(nn.Module):
    def __init__(self, input_size, hidden_units, output_size, hidden_layers):
        super(SIREN, self).__init__()
        self.input_size = input_size
        self.hidden_units = hidden_units
        self.output_size = output_size
        self.hidden_layers = hidden_layers

        layers = []
        layers.append(nn.Linear(self.input_size, self.hidden_units))
        layers.append(nn.SiLU()) # Sin activation

        for _ in range(self.hidden_layers - 1):
            layers.append(nn.Linear(self.hidden_units, self.hidden_units))
            layers.append(nn.SiLU())

        layers.append(nn.Linear(self.hidden_units, self.output_size))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
def train_siren(siren, Xf, Xg, k, analytical_solution, epochs=50000, lr=1e-3, stop_criterion=2e-16):
    optimizer = optim.LBFGS(siren.parameters(), lr=lr, max_iter=50)

    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = loss_function(siren, Xf, Xg, k, analytical_solution)
            loss.backward()
            return loss

        optimizer.step(closure)

        # Check stopping criterion
        gradients = torch.cat([param.grad.view(-1) for param in siren.parameters()])
        max_gradient = torch.max(torch.abs(gradients)).item()
        if max_gradient < stop_criterion:
            break
        
#         if epoch % 1000 == 0:
#             loss = loss_function(tann, Xf, Xg, k, analytical_solution)
#             print("Epoch:", epoch, "Loss:", loss.item())

    return siren


## PWNN

In [ ]:
#Custom activation function
class ExpI(nn.Module):
    def forward(self, x):
#         x_real = x[:, 0]
#         x_imag = x[:, 1]
#         y_real = torch.exp(x_real) * torch.cos(x_imag)
#         y_imag = torch.exp(x_real) * torch.sin(x_imag)
#         y = torch.cat((y_real, y_imag), dim=1)
        y = torch.exp(x)
        return y

In [ ]:
class PWNN(nn.Module):
    def __init__(self, input_size, hidden_units, output_size, hidden_layers):
        super(PWNN, self).__init__()
        self.input_size = input_size
        self.hidden_units = hidden_units
        self.output_size = output_size
        self.hidden_layers = hidden_layers

        layers = []
        layers.append(nn.Linear(self.input_size, self.hidden_units))
        layers.append(ExpI()) # Real part activation function
        layers.append(nn.Linear(self.hidden_units, self.hidden_units))
        layers.append(ExpI()) # Real part activation function

        for _ in range(self.hidden_layers - 1):
            layers.append(nn.Linear(self.hidden_units, self.hidden_units))
            layers.append(ExpI()) # Real part activation function

        layers.append(nn.Linear(self.hidden_units, self.output_size))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        y = self.model(x)

        return y

In [ ]:
def train_pwnn(pwnn, Xf, Xg, k, analytical_solution, epochs=50000, lr=1e-3, stop_criterion=2e-16):
    optimizer = optim.LBFGS(pwnn.parameters(), lr=lr, max_iter=50)

    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = loss_function_pwnn(pwnn, Xf, Xg, k, analytical_solution)
            loss.backward()
            return loss

        optimizer.step(closure)

        # Check stopping criterion
        gradients = torch.cat([param.grad.view(-1) for param in pwnn.parameters()])
        max_gradient = torch.max(torch.abs(gradients)).item()
        if max_gradient < stop_criterion:
            break
        
#         if epoch % 1000 == 0:
#             loss = loss_function_pwnn(pwnn, Xf, Xg, k, analytical_solution)
#             print("Epoch:", epoch, "Loss:", loss.item())

    return pwnn

## Experiments

In [ ]:
def create_test_data(N_test, x_min, x_max, y_min, y_max):
    X_test = np.linspace(x_min, x_max, int(np.sqrt(N_test)))
    Y_test = np.linspace(y_min, y_max, int(np.sqrt(N_test)))
    X_test, Y_test = np.meshgrid(X_test, Y_test)
    X_test = X_test.flatten()
    Y_test = Y_test.flatten()
    X_test = np.column_stack((X_test, Y_test))
    X_test = torch.tensor(X_test, dtype=torch.float32)
    return X_test

def calculate_accuracy(trained_tann, X_test, k, analytical_solution):
    with torch.no_grad():
        X_test.requires_grad = True
        u_pred = trained_tann(X_test)
        print(type(u_pred))
        u_exact = analytical_solution(X_test[:, 0].numpy(), X_test[:, 1].numpy(), k)
        u_exact = np.real(u_exact)
        u_exact = torch.tensor(u_exact, dtype=torch.float32).view(-1, 1)
        error = torch.norm(np.abs(u_exact) - np.abs(u_pred), dim=0) / torch.norm(np.abs(u_exact), dim=0)
        accuracy = -torch.log10(error)
        if(accuracy<0):
            print("error:",error,"acc:", accuracy)
        return accuracy.item()

In [ ]:
# Parameters
k = 20
Ng = 5*k*4
Nf = 5*(k**2)
Layers = [1,2,3,4]
Units = [k,2*k,4*k]

x_min,x_max,y_min,y_max = 0,1,-0.5,0.5
epochs = 2000
lr= 1e-3
input_size = 2
output_size = 1
N_test = 10000

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print("Pwnn model with", Layers, Units)
for layer in Layers:
    for unit in Units:
        # Create training data
        Xf,Xg = create_training_data(Nf, Ng, x_min, x_max, y_min, y_max)
        print(Xf[0])
        # Create Model
        # model = TANN(input_size, unit, output_size, layer)
        # model = SIREN(input_size, unit, output_size, layer)
        model = PWNN(input_size, unit, output_size, layer)
        trained_model = train_tann(model, Xf, Xg, k, analytical_solution, epochs=epochs, lr=lr)
        # Create Test Data
        X_test = create_test_data(N_test, x_min, x_max, y_min, y_max)
        accuracy = calculate_accuracy(trained_model, X_test, k, analytical_solution)
        print("Layers:",layer,"Units:",unit,"Accuracy: ", accuracy)